# ClickHouse Integration with Data Lake

This notebook demonstrates how to connect ClickHouse to the MinIO data lake.

## 1. Install ClickHouse Client

In [ ]:
!pip install -q clickhouse-connect

## 2. ClickHouse SQL Examples

### Reading Parquet from MinIO (S3-compatible)

```sql
-- Create table using S3 table function
SELECT *
FROM s3(
    'http://minio:9000/warehouse/data/orders.parquet',
    'admin',
    'password123',
    'Parquet'
)
LIMIT 10;
```

### Create External Table

```sql
-- Create table with S3 engine
CREATE TABLE orders_from_s3
(
    order_id UInt32,
    customer_id UInt32,
    product_name String,
    quantity UInt16,
    unit_price Float64,
    total_amount Float64,
    order_date Date,
    status String
)
ENGINE = S3(
    'http://minio:9000/warehouse/data/orders.parquet',
    'admin',
    'password123',
    'Parquet'
);

-- Query the table
SELECT 
    status,
    count() as order_count,
    sum(total_amount) as revenue
FROM orders_from_s3
GROUP BY status
ORDER BY revenue DESC;
```

## 3. Using ClickHouse Python Client

In [ ]:
import clickhouse_connect

# Note: Update these settings based on your ClickHouse instance
# This example assumes ClickHouse is running externally
client = clickhouse_connect.get_client(
    host='localhost',
    port=8123,
    username='default',
    password=''
)

print("✓ Connected to ClickHouse")

### Query S3 Data Directly

In [ ]:
# Query data from MinIO via S3 function
query = """
SELECT *
FROM s3(
    'http://host.docker.internal:9000/warehouse/data/orders.parquet',
    'admin',
    'password123',
    'Parquet'
)
LIMIT 10
"""

result = client.query(query)
print(result.result_set)

### Convert to Pandas DataFrame

In [ ]:
import pandas as pd

# Execute query and get as DataFrame
df = client.query_df(
    """
    SELECT 
        product_name,
        count() as order_count,
        sum(total_amount) as total_revenue,
        avg(total_amount) as avg_order_value
    FROM s3(
        'http://host.docker.internal:9000/warehouse/data/orders.parquet',
        'admin',
        'password123',
        'Parquet'
    )
    GROUP BY product_name
    ORDER BY total_revenue DESC
    """
)

print("Product Revenue Analysis:")
df

## 4. Advanced: Reading JSON from S3

In [ ]:
# Query JSON data
customers_df = client.query_df(
    """
    SELECT *
    FROM s3(
        'http://host.docker.internal:9000/warehouse/data/customers.json',
        'admin',
        'password123',
        'JSONEachRow'
    )
    """
)

print("Customer Data:")
customers_df.head()

## 5. Join Data from Multiple S3 Sources

In [ ]:
# Join orders with customers
joined_df = client.query_df(
    """
    SELECT 
        c.name,
        c.city,
        count() as order_count,
        sum(o.total_amount) as total_spent
    FROM s3(
        'http://host.docker.internal:9000/warehouse/data/orders.parquet',
        'admin',
        'password123',
        'Parquet'
    ) AS o
    JOIN s3(
        'http://host.docker.internal:9000/warehouse/data/customers.json',
        'admin',
        'password123',
        'JSONEachRow'
    ) AS c ON o.customer_id = c.customer_id
    GROUP BY c.name, c.city
    ORDER BY total_spent DESC
    """
)

print("Customer Purchase Summary:")
joined_df

## 6. Creating Materialized Views

```sql
-- Create a local table
CREATE TABLE orders_local
(
    order_id UInt32,
    customer_id UInt32,
    product_name String,
    quantity UInt16,
    unit_price Float64,
    total_amount Float64,
    order_date Date,
    status String
)
ENGINE = MergeTree()
ORDER BY (order_date, order_id);

-- Load data from S3
INSERT INTO orders_local
SELECT *
FROM s3(
    'http://minio:9000/warehouse/data/orders.parquet',
    'admin',
    'password123',
    'Parquet'
);
```

## 7. Connection Tips

### From Docker Container
If ClickHouse runs in Docker, use service names:
```
http://minio:9000/warehouse/...
```

### From Host Machine
If ClickHouse runs on host, use localhost:
```
http://localhost:9000/warehouse/...
```

### From Different Docker Network
Use host.docker.internal:
```
http://host.docker.internal:9000/warehouse/...
```